In [1]:
%matplotlib inline

In [2]:
from coqLink import *

import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, OrderedDict
from itertools import count
import numpy as np
import pdb

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

import os
import signal

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

#plt.ion()

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# n = number of atomic variables
# state: goal (one hot vector of length n) + 
#        atomic hypotheses vector (length n) + 
#        pairs of one hot vectors (length n) representing implications (padded to reach m pairs total)


# GENERATE DATA, I.E. VALID SENTENCES OF IMPLICATIONAL LOGIC

def theoremGenerate(numAtoms, numImplications, numNecessaryImplications = None):
    
    if numNecessaryImplications == None:
        numNecessaryImplications = np.random.randint(int(numImplications/2.), numNecessaryImplications)
        
    if numNecessaryImplications > numImplications:
        print("Too many necessary...")
        return
        
    variablePerm = np.random.permutation(numAtoms)
    
    # choose a random propositional goal
    goal = np.zeros(numAtoms)
    goal[0] = 1.
    goal = goal[variablePerm]
    
    # choose the initial atomic propositional hypothesis
    atomicHypotheses = np.zeros(numAtoms)
    atomicHypotheses[numNecessaryImplications] = 1.
    atomicHypotheses = atomicHypotheses[variablePerm]
    
    # no duplicates and no trivial implications (e.g. A -> A)
    implicationList = [(i+1,i) for i in range(numNecessaryImplications)]
    while len(implicationList) < numImplications:
        implicationList.append(tuple(np.random.choice(range(numAtoms), 2, replace=False)))
        implicationList = list(set(implicationList))
        
    allImplications = np.zeros((numImplications, 2 * numAtoms))
    
    for i, implicationTuple in enumerate(implicationList):
        leftIndex,rightIndex = implicationTuple
        left = np.zeros(numAtoms)
        left[leftIndex] = 1.
        left = left[variablePerm]
        
        right = np.zeros(numAtoms)
        right[rightIndex] = 1.
        right = right[variablePerm]
        allImplications[i,:] = np.hstack((left,right))
        
    
    np.random.shuffle(allImplications)
    allImplications = allImplications.flatten()
    validTheorem = np.hstack((goal, atomicHypotheses, allImplications))
    return validTheorem

Replay Memory
-------------

We'll be using experience replay memory for training our DQN. It stores
the transitions that the agent observes, allowing us to reuse this data
later. By sampling from it randomly, the transitions that build up a
batch are decorrelated. It has been shown that this greatly stabilizes
and improves the DQN training procedure.

For this, we're going to need two classses:

-  ``Transition`` - a named tuple representing a single transition in
   our environment
-  ``ReplayMemory`` - a cyclic buffer of bounded size that holds the
   transitions observed recently. It also implements a ``.sample()``
   method for selecting a random batch of transitions for training.




In [4]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

Now, let's define our model. But first, let quickly recap what a DQN is.

DQN algorithm
-------------

Our environment is deterministic, so all equations presented here are
also formulated deterministically for the sake of simplicity. In the
reinforcement learning literature, they would also contain expectations
over stochastic transitions in the environment.

Our aim will be to train a policy that tries to maximize the discounted,
cumulative reward
$R_{t_0} = \sum_{t=t_0}^{\infty} \gamma^{t - t_0} r_t$, where
$R_{t_0}$ is also known as the *return*. The discount,
$\gamma$, should be a constant between $0$ and $1$
that ensures the sum converges. It makes rewards from the uncertain far
future less important for our agent than the ones in the near future
that it can be fairly confident about.

The main idea behind Q-learning is that if we had a function
$Q^*: State \times Action \rightarrow \mathbb{R}$, that could tell
us what our return would be, if we were to take an action in a given
state, then we could easily construct a policy that maximizes our
rewards:

\begin{align}\pi^*(s) = \arg\!\max_a \ Q^*(s, a)\end{align}

However, we don't know everything about the world, so we don't have
access to $Q^*$. But, since neural networks are universal function
approximators, we can simply create one and train it to resemble
$Q^*$.

For our training update rule, we'll use a fact that every $Q$
function for some policy obeys the Bellman equation:

\begin{align}Q^{\pi}(s, a) = r + \gamma Q^{\pi}(s', \pi(s'))\end{align}

The difference between the two sides of the equality is known as the
temporal difference error, $\delta$:

\begin{align}\delta = Q(s, a) - (r + \gamma \max_a Q(s', a))\end{align}

To minimise this error, we will use the `Huber
loss <https://en.wikipedia.org/wiki/Huber_loss>`__. The Huber loss acts
like the mean squared error when the error is small, but like the mean
absolute error when the error is large - this makes it more robust to
outliers when the estimates of $Q$ are very noisy. We calculate
this over a batch of transitions, $B$, sampled from the replay
memory:

\begin{align}\mathcal{L} = \frac{1}{|B|}\sum_{(s, a, s', r) \ \in \ B} \mathcal{L}(\delta)\end{align}

\begin{align}\text{where} \quad \mathcal{L}(\delta) = \begin{cases}
     \frac{1}{2}{\delta^2}  & \text{for } |\delta| \le 1, \\
     |\delta| - \frac{1}{2} & \text{otherwise.}
   \end{cases}\end{align}

Q-network
^^^^^^^^^

Our model will be a convolutional neural network that takes in the
difference between the current and previous screen patches. It has two
outputs, representing $Q(s, \mathrm{left})$ and
$Q(s, \mathrm{right})$ (where $s$ is the input to the
network). In effect, the network is trying to predict the *quality* of
taking each action given the current input.




In [5]:
class DQN(nn.Module):

    def __init__(self, numAtoms, numImplications, hiddenSize):
        super(DQN, self).__init__()
        
        # n = number of atomic variables
        # state: goal (one hot vector of length n) + 
        #        atomic hypotheses vector (length n) + 
        #        pairs of one hot vectors (length n) representing implications (padded to reach m pairs total)
        inputLength = numAtoms + numAtoms + (2 * numAtoms) * numImplications
        numAllHypotheses = numAtoms + numImplications
        
        self.fc1 = nn.Linear(inputLength, hiddenSize)
        self.fc2 = nn.Linear(hiddenSize, hiddenSize)
        self.fc3 = nn.Linear(hiddenSize, numAllHypotheses + 1) # +1 for back step (undo last action)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x)
        
    
'''
class DQN(nn.Module):

    def __init__(self):
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=2)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=5, stride=2)
        self.bn3 = nn.BatchNorm2d(32)
        self.head = nn.Linear(448, 2)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        return self.head(x.view(x.size(0), -1))
''';

Training
--------

Hyperparameters and utilities
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
This cell instantiates our model and its optimizer, and defines some
utilities:

-  ``select_action`` - will select an action accordingly to an epsilon
   greedy policy. Simply put, we'll sometimes use our model for choosing
   the action, and sometimes we'll just sample one uniformly. The
   probability of choosing a random action will start at ``EPS_START``
   and will decay exponentially towards ``EPS_END``. ``EPS_DECAY``
   controls the rate of the decay.
-  ``plot_durations`` - a helper for plotting the durations of episodes,
   along with an average over the last 100 episodes (the measure used in
   the official evaluations). The plot will be underneath the cell
   containing the main training loop, and will update after every
   episode.




In [6]:
def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            action = policy_net(state).max(1)[1].view(1, 1)
            return action
    else:
        return torch.tensor([[np.random.randint(numAtoms + numImplications + 1)]], device=device, dtype=torch.long)


episode_durations = []


def plot_durations():
    plt.figure(2)
    plt.clf()
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        display.clear_output(wait=True)
        display.display(plt.gcf())

Training loop
^^^^^^^^^^^^^

Finally, the code for training our model.

Here, you can find an ``optimize_model`` function that performs a
single step of the optimization. It first samples a batch, concatenates
all the tensors into a single one, computes $Q(s_t, a_t)$ and
$V(s_{t+1}) = \max_a Q(s_{t+1}, a)$, and combines them into our
loss. By defition we set $V(s) = 0$ if $s$ is a terminal
state. We also use a target network to compute $V(s_{t+1})$ for
added stability. The target network has its weights kept frozen most of
the time, but is updated with the policy network's weights every so often.
This is usually a set number of steps but we shall use episodes for
simplicity.




In [7]:
def optimize_model():
    
    if len(memory) < BATCH_SIZE:
        return
    
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see http://stackoverflow.com/a/19343/3343043 for
    # detailed explanation).
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None, batch.next_state)), 
                                  device=device, dtype=torch.uint8)
    non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

Below, you can find the main training loop. At the beginning we reset
the environment and initialize the ``state`` Tensor. Then, we sample
an action, execute it, observe the next screen and the reward (always
1), and optimize our model once. When the episode ends (our model
fails), we restart the loop.

Below, `num_episodes` is set small. You should download
the notebook and run lot more epsiodes.




In [8]:
numAtoms = 5
numImplications = 4
numNecessaryImplications = 2
hiddenSize = 100

In [9]:
# CHECK DATA GENERATION IS CORRECT

def decomposeTheorem(codedTheorem, numAtoms):
    print("Goal: " + str([i for i in range(numAtoms) if codedTheorem[i] == 1][0]))
    print("Hypothesis: " + str([i - numAtoms for i in range(numAtoms, 2 * numAtoms) if codedTheorem[i] == 1][0]))
    print("Implications: ")
    
    for thisRow in codedTheorem[2 * numAtoms:].reshape((-1, numAtoms * 2)):
        left = thisRow[0:numAtoms]
        right = thisRow[numAtoms:]
        print(str([i for i in range(numAtoms) if left[i] == 1][0]) + "->" + str([i for i in range(numAtoms) if right[i] == 1][0]))
            

sampleTheorem = theoremGenerate(numAtoms, numImplications, numNecessaryImplications)
decomposeTheorem(sampleTheorem,numAtoms)

Goal: 4
Hypothesis: 2
Implications: 
2->1
1->4
1->3
3->0


In [10]:
def initCoq(theorem, numAtoms, process):
    
    resultDict = OrderedDict()
    variableString = "Variables %s: Prop." % " ".join(['V%s' % str(i) for i in range(numAtoms)])
    resultDict = doAdd(variableString, resultDict, process, debugList = [])
    
    goalIndex = str([i for i in range(numAtoms) if theorem[i] == 1][0])
    goalString = "Goal V%s." % goalIndex
    resultDict = doAdd(goalString, resultDict, process, debugList = [])
    
    initialAtomicHypothesis = str([i - numAtoms for i in range(numAtoms, 2 * numAtoms) if theorem[i] == 1][0])
    initialString = "Hypothesis A%s: V%s." % (initialAtomicHypothesis, initialAtomicHypothesis)
    resultDict = doAdd(initialString, resultDict, process, debugList = [])
    atomicHypothesisList = [int(initialAtomicHypothesis)]
    
    hypothesisIndex = 0
    for thisRow in theorem[2 * numAtoms:].reshape((-1, numAtoms * 2)):
        leftIndex = str([i for i in range(numAtoms) if thisRow[0:numAtoms][i] == 1][0])
        rightIndex = str([i for i in range(numAtoms) if thisRow[numAtoms:][i] == 1][0])
        implicationString = "Hypothesis I%s: V%s -> V%s." % (str(hypothesisIndex), leftIndex, rightIndex)
        resultDict = doAdd(implicationString, resultDict, process, debugList = [])
        hypothesisIndex += 1
    
    return resultDict, atomicHypothesisList

In [11]:
# rewards
errorReward = -5.
stepReward = -1.
qedReward = 50.

In [12]:
def doCoqStep(action, resultDict, numAtoms, numImplications, atomicHypothesisList, state, process):
        
    if action < numAtoms:
        if action not in atomicHypothesisList:
            reward = errorReward
            done = False
            next_state = state
            return reward, done, next_state, resultDict, atomicHypothesisList
        else:
            coqString = 'apply A%s.' % str(action)
            actionCode = 0
    elif action < numAtoms + numImplications:
        coqString = 'apply I%s.' % str(action - numAtoms)
        actionCode = 1
    else:
        thisID = list(resultDict.values())[-1][0]
        doCancel(thisID, process)
        _,_ = resultDict.popitem()
        key, value = resultDict.popitem()
        
        thisID, hypGoals, reward, done, next_state, atomicHypothesisList = value
        resultDict[key] = (thisID, hypGoals, reward, done, next_state, atomicHypothesisList)
        return reward, done, next_state, resultDict, atomicHypothesisList
    
    oldStepCount = len(resultDict)
    
    resultDict = doAdd(coqString, resultDict, process, debugList = [])
    
    resultDictValues = list(resultDict.values())
    hypothesisList, goal = resultDictValues[-1][1][0]
    
    if len(resultDict) == oldStepCount: # ERROR
        reward = errorReward
        done = False
        next_state = state
    elif goal == None: # THEOREM PROVED
        reward = qedReward
        done = True
        next_state = None
    else:    # STILL NOT DONE
        reward = stepReward
        done = False
        if actionCode == 1:
            next_state = state.clone().data.numpy()
            goalIndex = int(goal[1:])
            next_state[0,0:numAtoms] = np.zeros(numAtoms)
            next_state[0,goalIndex] = 1.
            next_state = torch.from_numpy(next_state)
            
            hypothesisList = [i for i in hypothesisList if i != 'none']
            for i in hypothesisList:
                next_state[int(i[1:])] = 1.
        else:
            print("weird state")
            next_state = state
    
    key,value = resultDict.popitem()
    resultDict[key] = (value[0], value[1], reward, done, next_state, atomicHypothesisList)
    return reward, done, next_state, resultDict, atomicHypothesisList

In [13]:
BATCH_SIZE = 3
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10

policy_net = DQN(numAtoms, numImplications, hiddenSize).to(device)
target_net = DQN(numAtoms, numImplications, hiddenSize).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters())
memory = ReplayMemory(10000)

steps_done = 0

In [14]:
def coqLoad():
    
    process = Popen(["/home/ubuntu/.opam/4.06.1/bin/sertop"], stdin=PIPE, stdout=PIPE, shell=True)
    
    outputList = []
    
    line = process.stdout.readline()
    outputList.append(line)
    while '(Feedback((doc_id 0)(span_id 1)(route 0)(contents Processed)))' not in line.decode('ASCII'):
        line = process.stdout.readline()
        outputList.append(line)
    process.terminate()
    return outputList


In [18]:
num_episodes = 1

for i_episode in range(num_episodes):
    
    
    try:
        p.terminate()
    except:
        pass
    '''
    
    if i_episode % 1 == 0:
        print("episode: " + str(i_episode))

    p = Popen(["/home/ubuntu/.opam/4.06.1/bin/sertop"], stdin=PIPE, stdout=PIPE, shell=True)
    nbsr = NonBlockingStreamReader(p.stdout)

    load = output_from_command_orig(p, nbsr, None) # omits Coq initialization data from results
    p.terminate()
    
    '''
    load = coqLoad()
    
    p = Popen(["/home/ubuntu/.opam/4.06.1/bin/sertop"], stdin=PIPE, stdout=PIPE, shell=True) 

    state = theoremGenerate(numAtoms, numImplications, numNecessaryImplications)
    resultDict, atomicHypothesisList = initCoq(state, numAtoms, p)
    
    numInitialHypothesis = len(resultDict)
    state = torch.from_numpy(state).float().unsqueeze(0)
    

    
    for t in count(): 
            
        # Select an action
        action = select_action(state)
        actionCode = action.item()
        
        # don't permit cancel to erase initial hypotheses
        if (len(resultDict) < numInitialHypothesis + 1) and (actionCode == numAtoms + numImplications):
            action = torch.tensor([[np.random.randint(numAtoms + numImplications)]], device=device, dtype=torch.long)
            actionCode = action.item()
            
        
        reward, done, next_state, resultDict, atomicHypothesisList = doCoqStep(actionCode, 
                                                                               resultDict, 
                                                                               numAtoms, 
                                                                               numImplications, 
                                                                               atomicHypothesisList, 
                                                                               state, 
                                                                               p)
        reward = torch.tensor([reward], device=device)
        
        # Store the transition in memory
            
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the target network)
        optimize_model()
        if done:
            episode_durations.append(t + 1)
            #plot_durations()
            break
    # Update the target network
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print("Goal dictionary: ")
pretty(resultDict)
p.kill()

print('Complete')
#plt.ioff()
#plt.show()

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


ValueError: not enough values to unpack (expected 6, got 2)

In [19]:
debug

> <ipython-input-12-10874467a547>(21)doCoqStep()
     19         key, value = resultDict.popitem()
     20 
---> 21         thisID, hypGoals, reward, done, next_state, atomicHypothesisList = value
     22         resultDict[key] = (thisID, hypGoals, reward, done, next_state, atomicHypothesisList)
     23         return reward, done, next_state, resultDict, atomicHypothesisList

ipdb> up
> <ipython-input-18-a5572d8e81d9>(52)<module>()
     50                                                                                atomicHypothesisList,
     51                                                                                state,
---> 52                                                                                p)
     53         reward = torch.tensor([reward], device=device)
     54 

ipdb> l 1,60
      1 num_episodes = 1
      2 
      3 for i_episode in range(num_episodes):
      4 
      5 
      6     try:
      7         p.terminate()
      8     except:
      9         pa

ipdb> actionCode
9
ipdb> (len(resultDict) < numInitialHypothesis + 1) and (actionCode == numAtoms + numImplications)
True
ipdb> down
> <ipython-input-12-10874467a547>(21)doCoqStep()
     19         key, value = resultDict.popitem()
     20 
---> 21         thisID, hypGoals, reward, done, next_state, atomicHypothesisList = value
     22         resultDict[key] = (thisID, hypGoals, reward, done, next_state, atomicHypothesisList)
     23         return reward, done, next_state, resultDict, atomicHypothesisList

ipdb> l 1,50
      1 def doCoqStep(action, resultDict, numAtoms, numImplications, atomicHypothesisList, state, process):
      2 
      3     if action < numAtoms:
      4         if action not in atomicHypothesisList:
      5             reward = errorReward
      6             done = False
      7             next_state = state
      8             return reward, done, next_state, resultDict, atomicHypothesisList
      9         else:
     10             coqString = 'apply A%s.' %

ipdb> pretty(resultDict)
Variables V0 V1 V2 V3 V4: Prop.
	('2', [(['none'], None)])
Goal V1.
	('3', [(['none'], 'V1')])
Hypothesis A0: V0.
	('5', [(['none'], 'V1')])
Hypothesis I0: V4 -> V3.
	('7', [(['none'], 'V1')])
Hypothesis I1: V0 -> V3.
	('9', [(['none'], 'V1')])
Hypothesis I2: V0 -> V1.
	('11', [(['none'], 'V1')])
ipdb> up
> <ipython-input-18-a5572d8e81d9>(52)<module>()
     50                                                                                atomicHypothesisList,
     51                                                                                state,
---> 52                                                                                p)
     53         reward = torch.tensor([reward], device=device)
     54 

ipdb> pretty(resultDict)
Variables V0 V1 V2 V3 V4: Prop.
	('2', [(['none'], None)])
Goal V1.
	('3', [(['none'], 'V1')])
Hypothesis A0: V0.
	('5', [(['none'], 'V1')])
Hypothesis I0: V4 -> V3.
	('7', [(['none'], 'V1')])
Hypothesis I1: V0 -> V3.
	('9', [(['